# CCS reference dataset 

This file aims at creating the reference dataset for North Pacific OSSE. It conctenates the variables form **/MITgcm_filtered_final** and **/MITgcm_it** folders. 

In [1]:
import xarray as xr
import numpy as np 
import glob
#for parallelizing
from joblib import Parallel
from joblib import delayed as jb_delayed

Choice of the OSSE zone between *crossover_CCS*, *crossover_centerpacific* and *crossover_hawaii*.

In [2]:
#zone = "crossover_CCS"
#zone = "crossover_centerpacific"
# zone = "crossover_hawaii"
zone = "hawaii"


Setting the min and max coordinates 

In [3]:
if zone == "hawaii":
    lon_min=185;lon_max=205
    lat_min=15;lat_max=35
    

### 1) - Data Loading 

Total SSH and Balanced Motion SSH : 

In [4]:
# list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201205*.nc") + \
#              glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201206*.nc") + \
#              glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201207*.nc")
list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201208*.nc") + \
             glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201209*.nc") + \
             glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201210*.nc")
# list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201206*.nc") 
list_files.sort()

n_try = 0
while n_try<10:
    try : 
        ds = xr.open_mfdataset(list_files,combine='nested',concat_dim='time',parallel=True)
        break
    except : 
        print("Opening netcdf file failed, trying again...")
    n_try+=1

ds_ssh = xr.open_mfdataset(list_files,parallel=True,combine='nested',concat_dim='time',drop_variables=["mask", "ssh_igw","ssh_bar","ssh_hf","ssh_dedac"])
ds_ssh

<xarray.Dataset> Size: 177GB
Dimensions:    (time: 2112, latitude: 1681, longitude: 3121)
Coordinates:
  * latitude   (latitude) float32 7kB 10.0 10.02 10.04 ... 44.96 44.98 45.0
  * longitude  (longitude) float32 12kB 180.0 180.0 180.0 ... 245.0 245.0 245.0
  * time       (time) datetime64[ns] 17kB 2012-08-01 ... 2012-10-27T23:00:00
Data variables:
    ssh_bm     (time, latitude, longitude) float64 89GB dask.array<chunksize=(3, 241, 446), meta=np.ndarray>
    ssh_cor    (time, latitude, longitude) float64 89GB dask.array<chunksize=(24, 1681, 3121), meta=np.ndarray>
Attributes:
    NCO:      netCDF Operators version 5.0.6 (Homepage = http://nco.sf.net, C...
    history:  Tue Oct  8 10:20:18 2024: ncatted -a eulaVlliF_,latitude,d,, -a...

Internal tide SSH : 

In [5]:
# list_files = glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201205*.nc") + \
#              glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201206*.nc") + \
#              glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201207*.nc") 
list_files = glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201208*.nc") + \
             glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201209*.nc") + \
             glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201210*.nc") 
# list_files = glob.glob(f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/MITgcm_it_201206*.nc") 
ds_it = xr.open_mfdataset(list_files,parallel=True,combine='nested',concat_dim='time',drop_variables=["ssh_it2", "ssh_it3","ssh_it_tot"])
ds_it 

<xarray.Dataset> Size: 16GB
Dimensions:    (time: 2112, latitude: 961, longitude: 961)
Coordinates:
  * latitude   (latitude) float32 4kB 15.0 15.02 15.04 ... 34.96 34.98 35.0
  * longitude  (longitude) float32 4kB 185.0 185.0 185.0 ... 205.0 205.0 205.0
  * time       (time) datetime64[ns] 17kB 2012-08-11 ... 2012-10-21T23:00:00
Data variables:
    ssh_it1    (time, latitude, longitude) float64 16GB dask.array<chunksize=(24, 961, 961), meta=np.ndarray>

### 2) - Data Processing 

Croping the **ds_ssh** dataset to only keep the CSS region :

In [6]:
ds_ssh = ds_ssh.sel(latitude=slice(lat_min,lat_max),
                      longitude=slice(lon_min,lon_max),drop=True)

Concatenating the datasets to create the reference dataset : 

In [7]:
dc_ref = xr.merge([ds_ssh,ds_it])

del ds_ssh, ds_it

Renaming the variables of the reference dataset : 
- *ssh* for *ssh_cor* 
- *ssh_it* for *ssh_it1* 

In [8]:
dc_ref = dc_ref.rename({"ssh_cor":"ssh","ssh_it1":"ssh_it"})

Updating the attributes of each of the variable, including time coordinate 

In [9]:
dc_ref["ssh"] = dc_ref.ssh.assign_attrs({"description":"Reference Sea Surface Height (without barotropic tide).",
                                         "units":"[m]"})
dc_ref["ssh_bm"] = dc_ref.ssh_bm.assign_attrs({"description":"Balanced Motion signal of the reference Sea Surface Height.",
                                         "units":"[m]"})
dc_ref["ssh_it"] = dc_ref.ssh_it.assign_attrs({"description":"First Mode Internal Tide signal of the reference Sea Surface Height.",
                                         "units":"[m]"})

Optional : coarsening the resolution of the dataset to reduce the storage memory.

In [10]:
d_lon = 1/16 ; d_lat = 1/16
target_longitude = np.linspace(lon_min, lon_max,int((lon_max-lon_min+d_lon)/d_lon))
target_latitutde = np.linspace(lat_min, lat_max,int((lat_max-lat_min+d_lat)/d_lat))

In [11]:
dc_ref_coarse = True 

d_lon = 1/16 ; d_lat = 1/16
target_longitude = np.linspace(lon_min, lon_max,int((lon_max-lon_min+d_lon)/d_lon))
target_latitude = np.linspace(lat_min, lat_max,int((lat_max-lat_min+d_lat)/d_lat))

if dc_ref_coarse:

    # dc_ref_coarse = dc_ref.coarsen(longitude=2,latitude=2,boundary='trim',side = "right").mean()
    # dc_ref_coarse["latitude"] = dc_ref.latitude[:-1:2]
    # dc_ref_coarse["longitude"] = dc_ref.longitude[:-1:2]

    dc_ref_coarse = dc_ref.interp({"longitude":target_longitude,"latitude":target_latitude},method='linear')


### 3) - Data Saving 

In [12]:
name_folder = dict({"crossover_CCS":"2023b_SSHmapping_HF_California",
                    "crossover_centerpacific":"2023c_SSHmapping_HF_CenterPacific",
                    "crossover_hawaii":"2023d_SSHmapping_HF_Crossover_Hawaii",
                    "hawaii":"2023e_SSHmapping_HF_Hawaii"})

def save_date(dc_ref,date):
    if dc_ref_coarse : 
        dc_ref_coarse.sel(time=slice(np.datetime64(date.astype('str')+"T00"),np.datetime64(date.astype('str')+"T23")))\
            .to_netcdf(f"/bettik/bellemva/ocean_data_challenge/{name_folder[zone]}/dc_ref_eval_coarse/{name_folder[zone]}_eval_"+date.astype('str')+".nc")
    else : 
        dc_ref.sel(time=slice(np.datetime64(date.astype('str')+"T00"),np.datetime64(date.astype('str')+"T23")))\
            .to_netcdf(f"/bettik/bellemva/ocean_data_challenge/{name_folder[zone]}/dc_ref_eval/{name_folder[zone]}_eval_"+date.astype('str')+".nc")
    return True

In [13]:
array_date = np.arange(np.datetime64("2012-08-01"),np.datetime64("2012-10-28"))
# array_date = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"))

for date in array_date :
    save_date(dc_ref,date)
    print(date.astype('str')," : ok !")

2012-08-01  : ok !
2012-08-02  : ok !
2012-08-03  : ok !
2012-08-04  : ok !
2012-08-05  : ok !
2012-08-06  : ok !
2012-08-07  : ok !
2012-08-08  : ok !
2012-08-09  : ok !
2012-08-10  : ok !
2012-08-11  : ok !
2012-08-12  : ok !
2012-08-13  : ok !
2012-08-14  : ok !
2012-08-15  : ok !
2012-08-16  : ok !
2012-08-17  : ok !
2012-08-18  : ok !
2012-08-19  : ok !
2012-08-20  : ok !
2012-08-21  : ok !
2012-08-22  : ok !
2012-08-23  : ok !
2012-08-24  : ok !
2012-08-25  : ok !
2012-08-26  : ok !
2012-08-27  : ok !
2012-08-28  : ok !
2012-08-29  : ok !
2012-08-30  : ok !
2012-08-31  : ok !
2012-09-01  : ok !
2012-09-02  : ok !
2012-09-03  : ok !
2012-09-04  : ok !
2012-09-05  : ok !
2012-09-06  : ok !
2012-09-07  : ok !
2012-09-08  : ok !
2012-09-09  : ok !
2012-09-10  : ok !
2012-09-11  : ok !
2012-09-12  : ok !
2012-09-13  : ok !
2012-09-14  : ok !
2012-09-15  : ok !
2012-09-16  : ok !
2012-09-17  : ok !
2012-09-18  : ok !
2012-09-19  : ok !
2012-09-20  : ok !
2012-09-21  : ok !
2012-09-22  

In [ ]:
save_date(dc_ref,np.datetime64("2012-06-01"))

In [ ]:
test = xr.open_dataset("/bettik/bellemva/ocean_data_challenge/2023e_SSHmapping_HF_Hawaii/dc_ref_eval_coarse/2023e_SSHmapping_HF_Hawaii_eval_2012-06-01.nc")

In [ ]:
test.ssh_bm[0,:,:].values

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,12))
plt.pcolormesh(test.ssh_bm[12,:,:])
plt.colorbar()